In [1]:
import torch as th
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import scipy.io as sio

from NN import *
from th_operator import calc_grad
from utils import print_model_layers

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

In [2]:
# Task parameters
u0 = 0.1 # inlet flow velocity
rho = 5000 # density
mu = 0.5 # viscosity

# Samples
num_points_per_step = 10000  # number of spatial points per time step
num_BC_points_per_step = 1000  # number of boundary condition points per time step
num_IC_points = 50000  # number of initial condition points

# Data boundary
## Domain
x_ini, x_f, y_ini, y_f = -4, 16, -4, 4

## Time
T = 20  # total time in seconds
Delta_t = 0.1  # time step in seconds
num_time_steps = int(T / Delta_t)  # number of time steps
time_steps = np.linspace(0, T, num=num_time_steps)

## Circle
Cx, Cy, r = 0, 0, 0.5


def generate_domain_points(num_points, x_range, y_range, time_steps, circle_center, circle_radius):
    """
    Generate spatial-temporal points within specified domain excluding a circular obstacle, for given time steps.

    Parameters:
    - num_points: Number of points to generate for each time step.
    - x_range: Tuple of (min_x, max_x) for x-coordinate range.
    - y_range: Tuple of (min_y, max_y) for y-coordinate range.
    - time_steps: Array of time steps.
    - circle_center: Tuple of (Cx, Cy), the center of the circular obstacle.
    - circle_radius: Radius of the circular obstacle.

    Returns:
    - Numpy array of spatial-temporal points with shape (num_points * len(time_steps), 3), where each row represents (x, y, t),
      excluding points inside the circular obstacle.
    """
    Cx, Cy = circle_center
    xyt_points = []

    for t in time_steps:
        for _ in range(num_points):
            while True:
                x = np.random.uniform(x_range[0], x_range[1])
                y = np.random.uniform(y_range[0], y_range[1])
                # Check if the point is outside the circular obstacle
                if (x - Cx)**2 + (y - Cy)**2 >= circle_radius**2:
                    xyt_points.append([x, y, t])
                    break

    return np.array(xyt_points, dtype=np.float32)

def generate_boundary_points(num_points, boundary_func, time_steps):
    xyt_points = []
    for t in time_steps:
        points = boundary_func(num_points)
        t_col = np.full((points.shape[0], 1), t)
        xyt = np.hstack((points, t_col))
        xyt_points.append(xyt)
    return np.vstack(xyt_points)

def circle_boundary(num_points):
    # Generate points for the circle boundary
    theta = np.random.uniform(0, 2*np.pi, num_points)
    x = Cx + 2*r * np.cos(theta)
    y = Cy + 2*r * np.sin(theta)
    return np.vstack((x, y)).T

def wall_boundary(num_points, x_range, y_value):
    # Generate points for wall boundaries (top or bottom)
    x = np.random.uniform(x_range[0], x_range[1], num_points)
    y = np.full(num_points, y_value)
    return np.vstack((x, y)).T

def inlet_outlet_boundary(num_points, y_range, x_value):
    # Generate points for inlet or outlet boundaries
    y = np.random.uniform(y_range[0], y_range[1], num_points)
    x = np.full(num_points, x_value)
    return np.vstack((x, y)).T

def generate_initial_conditions(num_points, x_range, y_range):
    x_initial = np.random.uniform(x_range[0], x_range[1], num_points)
    y_initial = np.random.uniform(y_range[0], y_range[1], num_points)
    t_initial = np.zeros(num_points)
    xyt_initial = np.stack((x_initial, y_initial, t_initial), axis=-1)
    return xyt_initial

def train_dataset():
    # Generating boundary points for each boundary condition and time step
    xyt_eqn = generate_domain_points(num_points_per_step, (x_ini, x_f), (y_ini, y_f), time_steps, (0, 0), 0.5)
    xyt_circle = generate_boundary_points(num_BC_points_per_step, circle_boundary, time_steps)
    xyt_w1 = generate_boundary_points(num_BC_points_per_step, lambda num_points: wall_boundary(num_points, (x_ini, x_f), y_ini), time_steps)
    xyt_w2 = generate_boundary_points(num_BC_points_per_step, lambda num_points: wall_boundary(num_points, (x_ini, x_f), y_f), time_steps)
    xyt_in = generate_boundary_points(num_BC_points_per_step, lambda num_points: inlet_outlet_boundary(num_points, (y_ini, y_f), x_ini), time_steps)
    xyt_out = generate_boundary_points(num_BC_points_per_step, lambda num_points: inlet_outlet_boundary(num_points, (y_ini, y_f), x_f), time_steps)
    xyt_initial = generate_initial_conditions(num_IC_points, (x_ini, x_f), (y_ini, y_f))

    # Combine all training points
    return {
        "eqn": xyt_eqn,
        "circle": xyt_circle,
        "w1": xyt_w1,
        "w2": xyt_w2,
        "in": xyt_in,
        "out": xyt_out,
        "initial": xyt_initial,
    }

x_train = train_dataset()
# # Save the training data
# mu_str = '{:04.1f}'.format(mu)
# file_path = f"./trainX__rho_{rho:04d}__mu_{mu_str}__.mat"
# sio.savemat(file_path, {
#     "eqn": xyt_eqn,
#     "circle": xyt_circle,
#     "w1": xyt_w1,
#     "w2": xyt_w2,
#     "in": xyt_in,
#     "out": xyt_out,
#     "initial": xyt_initial,
# })

In [3]:
print("eqn shape: ", x_train["eqn"].shape)
print("circle shape: ", x_train["circle"].shape)
print("w1 shape: ", x_train["w1"].shape)
print("w2 shape: ", x_train["w2"].shape)
print("in shape: ", x_train["in"].shape)
print("out shape: ", x_train["out"].shape)
print("initial shape: ", x_train["initial"].shape)

eqn shape:  (2000000, 3)
circle shape:  (200000, 3)
w1 shape:  (200000, 3)
w2 shape:  (200000, 3)
in shape:  (200000, 3)
out shape:  (200000, 3)
initial shape:  (50000, 3)


In [4]:
# 배치학습을 위한 데이터 로더 함수를 정의합니다.
def create_dataloader(x_data, batch_size, shuffle):
    dataset = TensorDataset(th.tensor(x_data, dtype=th.float32))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

def generate_combined_batch(dataloaders_iterators, batch_size_per_loader):
    batch_parts = {}
    for key, iterator in dataloaders_iterators.items():
        try:
            # 각 DataLoader의 iterator로부터 데이터 배치를 가져옴
            data, = next(iterator)
            batch_parts[key] = data[:batch_size_per_loader]
        except StopIteration:
            # 현재 DataLoader의 데이터가 끝에 도달했을 경우, iterator를 다시 시작
            dataloaders_iterators[key] = iter(dataloaders[key])
            data, = next(dataloaders_iterators[key])
            batch_parts[key] = data[:batch_size_per_loader]
    return batch_parts

dataloaders = {key: create_dataloader(x_train[key], 10000, True) for key in x_train}
dataloaders_iterators = {key: iter(loader) for key, loader in dataloaders.items()}


# for _ in range(200):  # 200번의 iteration으로 1 epoch을 수행
#     print("---")
#     combined_batch = generate_combined_batch(dataloaders_iterators, batch_size_per_loader=10000)
#     for k, v in combined_batch.items():
#         print(f"{k}: {v}")

In [5]:
# Task parameters
u0 = 0.1 # inlet flow velocity
rho = 5000 # density
mu = 0.5 # viscosity

# Samples
num_train_samples = 20000 # number of training samples
num_test_samples = 100 # number of test samples

# Data boundary
# Domain
x_f =16
x_ini=-4
y_f= 4
y_ini= -4
# Time
T = 20  # total time in seconds
Delta_t = 0.1  # time step in seconds
num_time_steps = int(T / Delta_t)  # number of time steps
# Circle
Cx = 0
Cy = 0
a = 1
b = 1

# create training input
# Eqauation collocation points
xyt_eqn = np.random.rand(num_train_samples, 2)
xyt_eqn[...,0] = (x_f - x_ini)*xyt_eqn[...,0] + x_ini
xyt_eqn[...,1] = (y_f - y_ini)*xyt_eqn[...,1] + y_ini

for i in range(num_train_samples):
    while (xyt_eqn[i, 0] - Cx)**2/a**2 + (xyt_eqn[i, 1] - Cy)**2/b**2 < 1:
        xyt_eqn[i, 0] = (x_f - x_ini) * np.random.rand(1, 1) + x_ini
        xyt_eqn[i, 1] = (y_f - y_ini) * np.random.rand(1, 1) + y_ini

# Boundary collocation points
# Circle
xyt_circle = np.random.rand(num_train_samples, 2)
xyt_circle[...,0] = 2*(a)*xyt_circle[...,0] +(Cx-a)
xyt_circle[0:num_train_samples//2,1] = b*(1 - (xyt_circle[0:num_train_samples//2,0]-Cx)**2 / a**2)**0.5 + Cy
xyt_circle[num_train_samples//2:,1] = -b*(1 - (xyt_circle[num_train_samples//2:,0]-Cx)**2 / a**2)**0.5 + Cy

# Wall top and bottom
xyt_w1 = np.random.rand(num_train_samples, 2)  # top-bottom boundaries
xyt_w1[..., 0] = (x_f - x_ini)*xyt_w1[...,0] + x_ini
xyt_w1[..., 1] =  y_ini          # y-position is 0 or 1

xyt_w2 = np.random.rand(num_train_samples, 2)  # top-bottom boundaries
xyt_w2[..., 0] = (x_f - x_ini)*xyt_w2[...,0] + x_ini
xyt_w2[..., 1] =  y_f

# Inlet and outlet
xyt_in = np.random.rand(num_train_samples, 2)
xyt_in[...,0] = x_ini

xyt_out = np.random.rand(num_train_samples, 2)  # left-right boundaries
xyt_out[..., 0] = x_f

x_train = [xyt_eqn, xyt_in, xyt_out, xyt_w1, xyt_w2, xyt_circle]

C:\Users\enomazosii\AppData\Local\Temp\ipykernel_5608\4233571621.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  xyt_eqn[i, 0] = (x_f - x_ini) * np.random.rand(1, 1) + x_ini
C:\Users\enomazosii\AppData\Local\Temp\ipykernel_5608\4233571621.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  xyt_eqn[i, 1] = (y_f - y_ini) * np.random.rand(1, 1) + y_ini


In [14]:
MLP = MultiLayerPerceptronClass(
    x_dim=3, y_dim=3,
    h_dim_list=[48,48,48,48],
    actv= th.nn.Tanh(),
    p_drop=0.0,
    batch_norm=False
)
print_model_layers(model=MLP, x_torch=th.rand(size=[16,3], dtype=th.float32))
device = th.device("cuda" if th.cuda.is_available() else "cpu")
MLP = MLP.to(device)
# MLP = th.nn.DataParallel(MLP)
pinn = PINN(network=MLP, rho=rho, mu=mu)

batch_size:[16]
[  ] layer:[          input] size:[          16x3]
[ 0] layer:[      linear_00] size:[         16x48] numel:[       768]
[ 1] layer:[        tanh_01] size:[         16x48] numel:[       768]
[ 2] layer:[      linear_02] size:[         16x48] numel:[       768]
[ 3] layer:[        tanh_03] size:[         16x48] numel:[       768]
[ 4] layer:[      linear_04] size:[         16x48] numel:[       768]
[ 5] layer:[        tanh_05] size:[         16x48] numel:[       768]
[ 6] layer:[      linear_06] size:[         16x48] numel:[       768]
[ 7] layer:[        tanh_07] size:[         16x48] numel:[       768]
[ 8] layer:[      linear_08] size:[          16x3] numel:[        48]


In [18]:
class L_BFGS_B:
    """
    Optimize the PyTorch model using L-BFGS-B algorithm.
    Attributes:
        model: optimization target model.
        samples: training samples.
    """

    def __init__(self, model, dataloaders_iterators, generate_func, batch_size=10000, epochs=20):
        """
        Args:
            model: optimization target model.
            x_train: training input samples as tensors.
            y_train: training target samples as tensors.
        """
        self.model = model
        self.dataloaders_iterators = dataloaders_iterators
        self.generate_combined_batch = generate_func
        self.batch_size = batch_size
        self.epochs = epochs

        self.optimizer = optim.LBFGS(
            params = self.model.parameters(),
            lr = 0.5,
            max_iter = 1000,
            max_eval = 1000,
            tolerance_grad = 1e-5,
            tolerance_change = th.finfo(float).eps,
            history_size=50,
            line_search_fn="strong_wolfe",
            )

    def evaluate(self):
        """
        Evaluate loss and gradients for the model.
        Returns:
            loss: the loss as a scalar tensor.
        """
        def closure():
            
            
            if th.is_grad_enabled():
                self.optimizer.zero_grad()
            
            combined_batch = self.generate_combined_batch(self.dataloaders_iterators, self.batch_size)
            for key in combined_batch:
                combined_batch[key] = combined_batch[key].to(device)
            
            PDE, BC, IC = self.model(combined_batch)
            
            # PDE: u_eqn, v_eqn
            # BC: uv_in, uv_w1, uv_w2, uv_circle # uv_out is not used
            # IC: u_initial, v_initial
            
            inlet = BC[0][..., 1].unsqueeze(-1) # y' = y - ymin / ymax - ymin
            inlet = 4*inlet*(1-inlet)
            zeros = th.zeros((self.batch_size, 1))
            zeros = zeros.to(device)
            inlet = th.cat([inlet, zeros], dim = -1)
            
            # LOSS
            criterion = nn.MSELoss()
            loss_eqn = criterion(PDE, th.zeros_like(PDE))
            loss_in = criterion(BC[0], inlet)
            loss_periodic = criterion(BC[1], BC[2])
            loss_circle = criterion(BC[3], th.zeros_like(BC[3]))
            loss_ic = criterion(IC, th.zeros_like(IC))
            total_loss = loss_eqn + ( loss_in + loss_periodic + loss_circle + loss_ic) / 4

            if total_loss.requires_grad:
                total_loss.backward()
            

            return total_loss
        
        return closure

    def fit(self):
        """
        Train the model using L-BFGS-B algorithm.
        Args:
            max_iter: Maximum number of iterations.
        """
        self.model.train()
        # Optimize
        for epoch in range(self.epochs):
            for iter in range(200):
                loss = self.optimizer.step(self.evaluate())
                print(f"{epoch:02d}:{iter:03d}:{loss}")   
        print('Optimization finished.')
        
    def predict(self, x):
        """
        Predict using the trained model.
        Args:
            x: Input data for prediction.
        Returns:
            predictions: Predicted values by the model.
        """
        self.model.eval()  # 모델을 평가 모드로 설정
        with th.no_grad():  # 그라디언트 계산 비활성화
            predictions = self.model(x)
        return predictions

In [19]:
lbfgs = L_BFGS_B(model=pinn, dataloaders_iterators=dataloaders_iterators, generate_func=generate_combined_batch, batch_size=10000, epochs=3)
lbfgs.fit()

00:000:0.00590389221906662
00:001:0.0060229552909731865
00:002:0.0061650583520531654
00:003:0.005948071368038654
00:004:0.0060457224026322365
00:005:0.005891289561986923
00:006:0.006130452733486891
00:007:0.006213209126144648
00:008:0.005910563748329878
00:009:0.006130942143499851
00:010:0.005759900435805321
00:011:0.005914062261581421
00:012:0.005897027440369129
00:013:0.006111622788012028
00:014:0.006087162997573614
00:015:0.0060966163873672485
00:016:0.005811913870275021
00:017:0.005841623060405254
00:018:0.005793738178908825
00:019:0.0057824840769171715
00:020:0.006016789469867945
00:021:0.005846207961440086
00:022:0.005848351866006851
00:023:0.005942717660218477
00:024:0.005894653033465147
00:025:0.005881688557565212
00:026:0.005664526484906673
00:027:0.00579722598195076
00:028:0.0061312150210142136
00:029:0.005791706498712301
00:030:0.005940423347055912
00:031:0.005971744190901518
00:032:0.006002441048622131
00:033:0.005775209981948137
00:034:0.0058482130989432335
00:035:0.006077

In [20]:
th.save(MLP.state_dict(), "./model_state_dict.pth")

---

In [21]:
# import torch as th
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset

# import numpy as np
# import scipy.io as sio

# from NN import *
# from th_operator import calc_grad
# from utils import print_model_layers

# import matplotlib.pyplot as plt
# from matplotlib.colors import Normalize

def contour(x, y, z, title, levels=100):
    """
    Contour plot.
    Args:
        x: x-array.
        y: y-array.
        z: z-array.
        title: title string.
        levels: number of contour lines.
    """

    # get the value range
    vmin = np.min(z)
    vmax = np.max(z)

    # plot a contour
    font1 = {'family':'serif','size':20}
    plt.contour(x, y, z, colors='k', linewidths=0.2, levels=levels)
    contour_filled = plt.contourf(x, y, z, cmap='rainbow', levels=levels, norm=Normalize(vmin=vmin, vmax=vmax))

    # Add the circle patch to the current axes without altering the axes limits
    circle = plt.Circle((0, 0), 1, color='black')
    plt.gca().add_patch(circle)

    plt.title(title, fontdict=font1)
    plt.xlabel("X", fontdict=font1)
    plt.ylabel("Y", fontdict=font1)
    plt.tick_params(axis='both', which='major', labelsize=15)

    # Add colorbar
    cbar = plt.colorbar(contour_filled, pad=0.03, aspect=25, format='%.0e')
    cbar.mappable.set_clim(vmin, vmax)
    cbar.ax.tick_params(labelsize=15)

In [42]:
# 도메인 및 시간 설정
x_min, x_max = -4, 16
y_min, y_max = -4, 4
t_min, t_max = 0, 20  # 테스트할 시간 범위
delta_t = 0.1  # 시간 단위

# 공간 그리드 포인트 수
num_x, num_y = 100, 100 # 공간 및 시간 축에 대한 포인트 수

# 공간 및 시간 축을 위한 그리드 생성
x = np.linspace(x_min, x_max, num_x)
y = np.linspace(y_min, y_max, num_y)
X, Y = np.meshgrid(x, y)
test_shape = X.shape
time_steps = np.arange(t_min, t_max + delta_t, delta_t)

# 초기화
test_xyt = np.empty((0, 3), dtype=np.float32)

# 각 시간 스텝에 대해 x, y 그리드와 t 값을 결합
for t in time_steps:
    T = np.full(X.shape, t)
    xyt = np.stack([X.ravel(), Y.ravel(), T.ravel()], axis=-1)
    test_xyt = np.vstack([test_xyt, xyt])
    
# PyTorch 텐서로 변환 및 디바이스 할당
test_xyt = th.tensor(test_xyt, dtype=th.float32)

print(test_xyt.shape)

torch.Size([2010000, 3])


In [51]:
for time_steps, xyt in enumerate(test_xyt.split(10000)):
    xx = xyt[..., 0].unsqueeze(-1).reshape(test_shape).numpy()
    yy = xyt[..., 1].unsqueeze(-1).reshape(test_shape).numpy()
    xyt = th.tensor(xyt, dtype=th.float32).to(device)
    MLP.eval()
    u_v_p, _ = MLP(xyt)
    u, v, p = [ u_v_p[..., i].reshape(test_shape) for i in range(u_v_p.shape[-1]) ]
    u = u.detach().cpu().numpy()
    v = v.detach().cpu().numpy()
    p = p.detach().cpu().numpy()
    # compute (u, v)
    u = u.reshape(test_shape)
    v = v.reshape(test_shape)
    p = p.reshape(test_shape)

    # plot test results
    fig = plt.figure(figsize=(15, 6))
    contour(xx, yy, p, f'p_{time_steps:03d}')
    plt.tight_layout()
    plt.savefig(f'./p/Pressure_timestep_{time_steps:03d}.png')
    plt.close(fig)

    fig = plt.figure(figsize=(15, 6))
    contour(xx, yy, u, f'u_{time_steps:03d}')
    plt.tight_layout()
    plt.savefig(f'./u/Vx_timestep_{time_steps:03d}.png')
    plt.close(fig)

    fig = plt.figure(figsize=(15, 6))
    contour(xx, yy, v, f'v_{time_steps:03d}')
    plt.tight_layout()
    plt.savefig(f'./v/Vy_timestep_{time_steps:03d}.png')
    plt.close(fig)

C:\Users\enomazosii\AppData\Local\Temp\ipykernel_5608\2733593040.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xyt = th.tensor(xyt, dtype=th.float32).to(device)


In [55]:
from PIL import Image
from pathlib import Path
from glob import glob

frames = sorted(glob("./p/./*.png"))
frame_one = frames[0]
frame_one.save("figures/convergence.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=1)

In [63]:
from PIL import Image
from pathlib import Path
import os

def make_gif(folder_path, title):
    frames = [Image.open(image) for image in sorted(glob(folder_path))]
    frame_one = frames[0]
    frame_one.save(f"./{title}.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=1)

In [65]:
make_gif("./p/*.png", "p")
make_gif("./u/*.png", "u")
make_gif("./v/*.png", "v")